## Lab 4: Deploy to Production - Use AgentCore Runtime with Observability

### Overview

In Lab 3 we scaled our Customer Support Agent by centralizing tools through AgentCore Gateway with secure authentication. Now we'll complete the production journey by deploying our agent to AgentCore Runtime with comprehensive observability. This will transform our prototype into a production-ready system that can handle real-world traffic with full monitoring and automatic scaling.

[Amazon Bedrock AgentCore Runtime](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/agents-tools-runtime.html) is a secure, fully managed runtime that empowers organizations to deploy and scale AI agents in production, regardless of framework, protocol, or model choice. It provides enterprise-grade reliability, automatic scaling, and comprehensive monitoring capabilities.

**Workshop Journey:**

- **Lab 1 (Done):** Create Agent Prototype - Built a functional customer support agent
- **Lab 2 (Done):** Enhance with Memory - Added conversation context and personalization
- **Lab 3 (Done):** Scale with Gateway & Identity - Shared tools across agents securely
- **Lab 4 (Current):** Deploy to Production - Used AgentCore Runtime with observability
- **Lab 5:** Build User Interface - Create a customer-facing application

### Why AgentCore Runtime & Production Deployment Matter

Current State (Lab 1-3): Agent runs locally with centralized tools but faces production challenges:

- Agent runs locally in a single session
- No comprehensive monitoring or debugging capabilities
- Cannot handle multiple concurrent users reliably

After this lab, we will have a production-ready agent infrastructure with:

- Serverless auto-scaling to handle variable demand
- Comprehensive observability with traces, metrics, and logging
- Enterprise reliability with automatic error recovery
- Secure deployment with proper access controls
- Easy management through AWS console and APIs and support for real-world production workloads.


### Adding comprehensive observability with AgentCore Observability

Additionally, AgentCore Runtime integrates seamlessly with [AgentCore Observability](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html) to provide full visibility into your agent's behavior in production. AgentCore Observability automatically captures traces, metrics, and logs from your agent interactions, tool usage, and memory access patterns. In this lab we will see how AgentCore Runtime integrates with CloudWatch GenAI Observability to provide comprehensive monitoring and debugging capabilities.

For request tracing, AgentCore Observability captures the complete conversation flow including tool invocations, memory retrievals, and model interactions. For performance monitoring, it tracks response times, success rates, and resource utilization to help optimize your agent's performance.

During the observability flow, AgentCore Runtime automatically instruments your agent code and sends telemetry data to CloudWatch. You can then use CloudWatch dashboards and GenAI Observability features to analyze patterns, identify bottlenecks, and troubleshoot issues in real-time.

### Architecture for Lab 4
<div style="text-align:left"> 
    <img src="images/architecture_lab4_runtime.png" width="75%"/> 
</div>

*Agent now runs in AgentCore Runtime with full observability through CloudWatch, serving production traffic with auto-scaling and comprehensive monitoring. Memory and Gateway integrations from previous labs remain fully functional in the production environment.*

### Key Features

- **Serverless Agent Deployment:** Transform your local agent into a scalable production service using AgentCore Runtime with minimal code changes
- **Comprehensive Observability:** Full request tracing, performance metrics, and debugging capabilities through CloudWatch GenAI Observability

### Prerequisites

- Python 3.12+
- AWS account with appropriate permissions
- Docker, Finch or Podman installed and running
- Amazon Bedrock AgentCore SDK
- Strands Agents framework
- **Lab 3 Completion:** This lab builds on Lab 3 (AgentCore Gateway). You MUST run [lab-03-agentcore-gateway](lab-03-agentcore-gateway.ipynb) to provision the gateway before running this lab.

**Note**: You MUST enable [CloudWatch Transaction Search](https://docs.aws.amazon.com/AmazonCloudWatch/latest/monitoring/Enable-TransactionSearch.html) to be able to see AgentCore Observability traces in CloudWatch.


### Step 1: Import Required Libraries

In [2]:
# Import required libraries
import boto3
from lab_helpers.utils import get_ssm_parameter
from lab_helpers.lab2_memory import create_or_get_memory_resource

create_or_get_memory_resource()  # Just in case the memory lab wasn't executed

'CustomerSupportMemory-c7SRUYD152'

### Step 2: Preparing Your Agent for AgentCore Runtime

#### Creating the Runtime-Ready Agent

Let's first define the necessary AgentCore Runtime components via Python SDK within our previous local agent implementation.

Observe the #### AGENTCORE RUNTIME - LINE 1 #### comments below to see where is the relevant deployment code added. You'll find 4 such lines that prepare the runtime-ready agent:

1. Import the Runtime App with `from bedrock_agentcore.runtime import BedrockAgentCoreApp`
2. Initialize the App with `app = BedrockAgentCoreApp()`
3. Decorate our invocation function with `@app.entrypoint`
4. Let AgentCore Runtime control the execution with `app.run()`

##### Key Implementation Details:

The runtime-ready agent uses an entrypoint function that extracts user prompts from the payload and JWT tokens from request headers via 
context.request_headers.get('Authorization', ''). The authorization token is then propagated directly to the AgentCore Gateway by passing it in the 
MCP client headers: headers={"Authorization": auth_header}. The implementation includes error handling for missing authentication and returns plain 
text responses from synchronous agent invocation while preserving all memory and tool functionality from previous labs.

In [3]:
%%writefile ./lab_helpers/lab4_runtime.py
from bedrock_agentcore.runtime import (
    BedrockAgentCoreApp,
)  #### AGENTCORE RUNTIME - LINE 1 ####
from strands import Agent
from strands.tools.mcp import MCPClient
from mcp.client.streamable_http import streamablehttp_client
import requests
import boto3
from strands.models import BedrockModel
from lab_helpers.utils import get_ssm_parameter
from lab_helpers.lab1_strands_agent import (
    get_return_policy,
    get_product_info,
    get_technical_support,
    SYSTEM_PROMPT,
    MODEL_ID,
)

from lab_helpers.lab2_memory import (
    CustomerSupportMemoryHooks,
    memory_client,
    ACTOR_ID,
    SESSION_ID,
)

# Initialize boto3 client
sts_client = boto3.client('sts')

# Get AWS account details
REGION = boto3.session.Session().region_name

# Lab1 import: Create the Bedrock model
model = BedrockModel(model_id=MODEL_ID)

# Lab2 import : Initialize memory via hooks
memory_id = get_ssm_parameter("/app/customersupport/agentcore/memory_id")
memory_hooks = CustomerSupportMemoryHooks(
    memory_id, memory_client, ACTOR_ID, SESSION_ID
)

# Initialize the AgentCore Runtime App
app = BedrockAgentCoreApp()  #### AGENTCORE RUNTIME - LINE 2 ####

@app.entrypoint  #### AGENTCORE RUNTIME - LINE 3 ####
async def invoke(payload, context=None):
    """AgentCore Runtime entrypoint function"""
    user_input = payload.get("prompt", "")

    # Access request headers - handle None case
    request_headers = context.request_headers or {}

    # Get Client JWT token
    auth_header = request_headers.get('Authorization', '')

    print(f"Authorization header: {auth_header}")
    # Get Gateway ID
    existing_gateway_id = get_ssm_parameter("/app/customersupport/agentcore/gateway_id")
    
    # Initialize Bedrock AgentCore Control client
    gateway_client = boto3.client(
        "bedrock-agentcore-control",
        region_name=REGION,
    )
    # Get existing gateway details
    gateway_response = gateway_client.get_gateway(gatewayIdentifier=existing_gateway_id)

    # Get gateway url
    gateway_url = gateway_response['gatewayUrl']

    # Create MCP client and agent within context manager if JWT token available
    if gateway_url and auth_header:
        try:
                mcp_client = MCPClient(lambda: streamablehttp_client(
                    url=gateway_url,
                    headers={"Authorization": auth_header}  
                ))
                
                with mcp_client:
                    #tools = mcp_client.list_tools_sync()
                    tools = (
                        [
                            get_product_info,
                            get_return_policy,
                            get_technical_support
                        ]
                        + mcp_client.list_tools_sync()
                    )
                    
                    # Create the agent with all customer support tools
                    agent = Agent(
                        model=model,
                        tools=tools,
                        system_prompt=SYSTEM_PROMPT,
                        hooks=[memory_hooks],
                    )
                    # Invoke the agent
                    response = agent(user_input)
                    return response.message["content"][0]["text"]
        except Exception as e:
                print(f"MCP client error: {str(e)}")
                return f"Error: {str(e)}"
    else:
        return "Error: Missing gateway URL or authorization header"

if __name__ == "__main__":
    app.run()  #### AGENTCORE RUNTIME - LINE 4 ####

Overwriting ./lab_helpers/lab4_runtime.py


#### What happens behind the scenes?

When you use `BedrockAgentCoreApp`, it automatically:

- Creates an HTTP server that listens on port 8080
- Implements the required `/invocations` endpoint for processing requests
- Implements the `/ping` endpoint for health checks
- Handles proper content types and response formats
- Manages error handling according to AWS standards


### Step 3: Deploying to AgentCore Runtime

Now let's deploy our agent to AgentCore Runtime using the [AgentCore Starter Toolkit](https://github.com/aws/bedrock-agentcore-starter-toolkit).

#### Configure the Secure Runtime Deployment (AgentCore Runtime + AgentCore Identity)

First we will use our starter toolkit to configure the AgentCore Runtime deployment with an entrypoint, the execution role we will create and a requirements file. We will also configure the identity authorization using an Amazon Cognito user pool and we will configure the starter kit to auto create the Amazon ECR repository on launch.

During the configure step, your docker file will be generated based on your application code

<div style="text-align:left"> 
    <img src="images/configure.png" width="75%"/> 
</div>

**Note**: The Cognito access_token is valid for 2 hours only. If the access_token expires you can vend another access_token by using the `reauthenticate_user` method.


In [4]:
from lab_helpers.utils import get_or_create_cognito_pool

access_token = get_or_create_cognito_pool(refresh_token=True)
print(f"Access token: {access_token['bearer_token']}")

Access token: eyJraWQiOiJObHRmcU1telFNa1czM0x1dUZZbjhTRkV4R29xVmM2WUdSdG1VVVN2UEpzPSIsImFsZyI6IlJTMjU2In0.eyJzdWIiOiIxNGU4YTRlOC0xMDIxLTcwNTgtNThiNi1jOTNiODI4ZWQ2NTIiLCJpc3MiOiJodHRwczpcL1wvY29nbml0by1pZHAudXMtZWFzdC0xLmFtYXpvbmF3cy5jb21cL3VzLWVhc3QtMV9Mdm5iOWRtNDIiLCJjbGllbnRfaWQiOiI3N3RoZWIybGEwczNzNTgxNnZpZjdtams4NyIsIm9yaWdpbl9qdGkiOiI0NmZjOWFhNi1jNWI1LTQ2ZjMtYTAzOS0yZmM1Mjc3NmVhNGQiLCJldmVudF9pZCI6IjAyNGUwMGU0LTgwNTgtNDdkOS05NTY1LTdmZmUwOThmNjRjZiIsInRva2VuX3VzZSI6ImFjY2VzcyIsInNjb3BlIjoiYXdzLmNvZ25pdG8uc2lnbmluLnVzZXIuYWRtaW4iLCJhdXRoX3RpbWUiOjE3NzAxNDE5NDcsImV4cCI6MTc3MDE0NTU0NywiaWF0IjoxNzcwMTQxOTQ3LCJqdGkiOiI0M2ZiOGZhNS03ZTI4LTRmODEtODc4Yi1hNTczMzY4NTcwMDAiLCJ1c2VybmFtZSI6InRlc3R1c2VyIn0.L9Xt6qCF8r-JR8oz2uRoFQaP02IN62i5Itxr5irS5HaVhK2QMClHJb1Jkbi2yAf0aNCRyECKGyDOag_Tw7NvRNrMUaJDPzyRa8nmiDLs6oIhFqgz0OHQ-SToV2RRfrItDd8oCzFzIFdEyqa0CQ36Y55XwANojHx4VnRZwjrMFMoepOwcy8S-AwFxyWuyp0FGPDLebWrdN_SYI1P5lFMmxVXxv5f3iFv2tA5byQT_FrZgyfhugC2zmcSpLGD1ImO4gWXjtpNLBpgqAxNB3F6dtoX5dGu6A_B58xGpuV

#### AgentCore Runtime Configuration Summary:

Below code configures the AgentCore Runtime deployment using the starter toolkit. It creates an execution role for the runtime, then configures the 
deployment with the agent entrypoint file (lab_helpers/lab4_runtime.py), enables automatic ECR repository creation, and sets up JWT-based authentication using 
Cognito. The configuration specifies allowed client IDs and discovery URLs retrieved from SSM parameters, establishing secure access control for the 
production agent deployment. This step automatically generates the Dockerfile and .bedrock_agentcore.yaml configuration files needed for 
containerized deployment.

**Runtime Header Configuration** : Below code configures custom header allowlists for the deployed AgentCore Runtime. It extracts the runtime ID from the agent ARN, retrieves the 
current runtime configuration to preserve existing settings, then updates the runtime with a request header allowlist that includes the Authorization
header (required for OAuth token propagation) and custom headers. This ensures JWT tokens and other necessary headers are properly forwarded from 
client requests to the agent runtime code.

In [5]:
from bedrock_agentcore_starter_toolkit import Runtime
from lab_helpers.utils import create_agentcore_runtime_execution_role

# Initialize the runtime toolkit
boto_session = boto3.session.Session()
region = boto_session.region_name

execution_role_arn = create_agentcore_runtime_execution_role()

agentcore_runtime = Runtime()

# Configure the deployment
response = agentcore_runtime.configure(
    entrypoint="lab_helpers/lab4_runtime.py",
    execution_role=execution_role_arn,
    auto_create_ecr=True,
    requirements_file="requirements.txt",
    region=region,
    agent_name="customer_support_agent",
    authorizer_configuration={
        "customJWTAuthorizer": {
            "allowedClients": [
                get_ssm_parameter("/app/customersupport/agentcore/client_id")
            ],
            "discoveryUrl": get_ssm_parameter(
                "/app/customersupport/agentcore/cognito_discovery_url"
            ),
        }
    },
    # Add custom header allowlist for Authorization and custom headers
    request_header_configuration={
        "requestHeaderAllowlist": [
            "Authorization",  # Required for OAuth propogation
            "X-Amzn-Bedrock-AgentCore-Runtime-Custom-H1",  # Custom header
        ]
    },
)

print("Configuration completed:", response)

Entrypoint parsed: file=/home/sagemaker-user/amazon-bedrock-agentcore/lab_helpers/lab4_runtime.py, bedrock_agentcore_name=lab4_runtime
Configuring BedrockAgentCore agent: customer_support_agent


ℹ️ Role CustomerSupportAssistantBedrockAgentCoreRole-us-east-1 already exists
Role ARN: arn:aws:iam::005185643085:role/CustomerSupportAssistantBedrockAgentCoreRole-us-east-1


Will create new memory with mode: STM_ONLY
Memory configuration: Short-term memory only


⚠️  [WARNING] Platform mismatch: Current system is 'linux/amd64' but Bedrock AgentCore requires 'linux/arm64'.
For deployment options and workarounds, see: 
https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/getting-started-custom.html

Generated Dockerfile: /home/sagemaker-user/amazon-bedrock-agentcore/Dockerfile
Generated .dockerignore: /home/sagemaker-user/amazon-bedrock-agentcore/.dockerignore
Keeping 'customer_support_agent' as default agent
Bedrock AgentCore configured: /home/sagemaker-user/amazon-bedrock-agentcore/.bedrock_agentcore.yaml


Configuration completed: config_path=PosixPath('/home/sagemaker-user/amazon-bedrock-agentcore/.bedrock_agentcore.yaml') dockerfile_path=PosixPath('/home/sagemaker-user/amazon-bedrock-agentcore/Dockerfile') dockerignore_path=PosixPath('/home/sagemaker-user/amazon-bedrock-agentcore/.dockerignore') runtime='Docker' region='us-east-1' account_id='005185643085' execution_role='arn:aws:iam::005185643085:role/CustomerSupportAssistantBedrockAgentCoreRole-us-east-1' ecr_repository=None auto_create_ecr=True memory_id=None


#### Launch the Agent

Now let's launch our agent to AgentCore Runtime. This will create an AWS CodeBuild pipeline, the Amazon ECR repository and the AgentCore Runtime components.

<div style="text-align:left"> 
    <img src="images/launch.png" width="100%"/> 
</div>

*Note: This step might fail if the agent with the same name already exists. If you want to overwrite the existing Runtime, use this instead:*

``` launch_result = agentcore_runtime.launch(auto_update_on_conflict=True)```


In [6]:
# Launch the agent (this will build and deploy the container)
from lab_helpers.utils import put_ssm_parameter

launch_result = agentcore_runtime.launch()
print("Launch completed:", launch_result.agent_arn)

agent_arn = put_ssm_parameter(
    "/app/customersupport/agentcore/runtime_arn", launch_result.agent_arn
)

🚀 CodeBuild mode: building in cloud (RECOMMENDED - DEFAULT)
   • Build ARM64 containers in the cloud with CodeBuild
   • No local Docker required
💡 Available deployment modes:
   • runtime.launch()                           → CodeBuild (current)
   • runtime.launch(local=True)                 → Local development
   • runtime.launch(local_build=True)           → Local build + cloud deploy (NEW)
Creating memory resource for agent: customer_support_agent
✅ MemoryManager initialized for region: us-east-1
Creating new STM-only memory...
Created memory: customer_support_agent_mem-k3d2NyGqvU
✅ New memory created: customer_support_agent_mem-k3d2NyGqvU (provisioning in background)
Starting CodeBuild ARM64 deployment for agent 'customer_support_agent' to account 005185643085 (us-east-1)
Setting up AWS resources (ECR repository, execution roles)...
Getting or creating ECR repository for agent: customer_support_agent
✅ ECR repository available: 005185643085.dkr.ecr.us-east-1.amazonaws.com/bedrock-

Repository doesn't exist, creating new ECR repository: bedrock-agentcore-customer_support_agent


Getting or creating CodeBuild execution role for agent: customer_support_agent
Role name: AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-aa96c3a063
Reusing existing CodeBuild execution role: arn:aws:iam::005185643085:role/AmazonBedrockAgentCoreSDKCodeBuild-us-east-1-aa96c3a063
Using dockerignore.template with 45 patterns for zip filtering
Uploaded source to S3: customer_support_agent/source.zip
Created CodeBuild project: bedrock-agentcore-customer_support_agent-builder
Starting CodeBuild build (this may take several minutes)...
Starting CodeBuild monitoring...
🔄 QUEUED started (total: 0s)
✅ QUEUED completed in 1.0s
🔄 PROVISIONING started (total: 1s)
✅ PROVISIONING completed in 8.3s
🔄 DOWNLOAD_SOURCE started (total: 9s)
✅ DOWNLOAD_SOURCE completed in 3.1s
🔄 PRE_BUILD started (total: 12s)
✅ PRE_BUILD completed in 1.0s
🔄 BUILD started (total: 13s)
✅ BUILD completed in 17.6s
🔄 POST_BUILD started (total: 31s)
✅ POST_BUILD completed in 12.4s
🔄 COMPLETED started (total: 43s)
✅ COMPLETED complet

Launch completed: arn:aws:bedrock-agentcore:us-east-1:005185643085:runtime/customer_support_agent-l62Ut2AD7M


#### Check Deployment Status

Let's wait for the deployment to complete:


In [7]:
# Check current status
status_response = agentcore_runtime.status()

if status_response.endpoint is None:
    print("Launching agent...")
    agentcore_runtime.launch()  # Try launch() instead of deploy()
    print("✅ Launch initiated!")
else:
    print(f"Already deployed. Status: {status_response.endpoint['status']}")

✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: customer_support_agent_mem-k3d2NyGqvU...
  ✅ Found memory: customer_support_agent_mem-k3d2NyGqvU
Retrieved Bedrock AgentCore status for: customer_support_agent


Already deployed. Status: READY


In [8]:
import time

# Wait for the agent to be ready
status_response = agentcore_runtime.status()
status = status_response.endpoint["status"]

end_status = ["READY", "CREATE_FAILED", "DELETE_FAILED", "UPDATE_FAILED"]
while status not in end_status:
    print(f"Waiting for deployment... Current status: {status}")
    time.sleep(10)
    status_response = agentcore_runtime.status()
    status = status_response.endpoint["status"]

print(f"Final status: {status}")

✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: customer_support_agent_mem-k3d2NyGqvU...
  ✅ Found memory: customer_support_agent_mem-k3d2NyGqvU
Retrieved Bedrock AgentCore status for: customer_support_agent


Final status: READY


### Step 4: Invoking Your Deployed Agent

Now that our agent is deployed and ready, let's test it with some queries. We invoke the agent with the right authorization token type. In out case it'll be Cognito access token. Copy the access token from the cell above

<div style="text-align:left"> 
    <img src="images/invoke.png" width="100%"/> 
</div>

#### Using the AgentCore Starter Toolkit

We can validate that the agent works using the AgentCore Starter Toolkit for invocation. The starter toolkit can automatically create a session id for us to query our agent. Alternatively, you can also pass the session id as a parameter during invocation. For demonstration purpose, we will create our own session id.


In [9]:
# Initialize the AgentCore Control client
client = boto3.client("bedrock-agentcore-control")

# Get agent ARN from status response
status_response = agentcore_runtime.status()

# Check what's available and extract runtime ID
if status_response.endpoint is None:
    print("❌ Agent not deployed yet. Run: agentcore_runtime.launch()")
else:
    print(f"Available keys: {status_response.endpoint.keys()}")
    # Try different possible key names
    agent_arn = (status_response.endpoint.get('endpointArn') or 
                 status_response.endpoint.get('agentArn') or 
                 status_response.endpoint.get('arn'))
    
    if agent_arn:
        runtime_id = agent_arn.split(":")[-1].split("/")[-1]
        print(f"Runtime ID: {runtime_id}")
    else:
        print(f"Full endpoint data: {status_response.endpoint}")

✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: customer_support_agent_mem-k3d2NyGqvU...
  ✅ Found memory: customer_support_agent_mem-k3d2NyGqvU
Retrieved Bedrock AgentCore status for: customer_support_agent


Available keys: dict_keys(['ResponseMetadata', 'liveVersion', 'agentRuntimeEndpointArn', 'agentRuntimeArn', 'status', 'createdAt', 'lastUpdatedAt', 'name', 'id'])
Full endpoint data: {'ResponseMetadata': {'RequestId': 'ea00f8a5-af11-4579-82bb-ae54c9ca6758', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Tue, 03 Feb 2026 18:09:42 GMT', 'content-type': 'application/json', 'content-length': '417', 'connection': 'keep-alive', 'x-amzn-requestid': 'ea00f8a5-af11-4579-82bb-ae54c9ca6758', 'x-amzn-remapped-x-amzn-requestid': '721d407e-6ca9-4259-9f81-1ce96c4baff5', 'x-amzn-remapped-content-length': '417', 'x-amzn-remapped-connection': 'keep-alive', 'x-amz-apigw-id': 'YN38DFRHoAMENkg=', 'x-amzn-trace-id': 'Root=1-698239e6-673579dd5f49492277485d77', 'x-amzn-remapped-date': 'Tue, 03 Feb 2026 18:09:42 GMT'}, 'RetryAttempts': 0}, 'liveVersion': '1', 'agentRuntimeEndpointArn': 'arn:aws:bedrock-agentcore:us-east-1:005185643085:runtime/customer_support_agent-l62Ut2AD7M/runtime-endpoint/DEFAULT', 'agent

In [10]:
# Initialize the AgentCore Control client
client = boto3.client("bedrock-agentcore-control")

# Get agent ARN from status response
status_response = agentcore_runtime.status()
agent_arn = status_response.endpoint['agentRuntimeArn']

# Extract runtime ID from the ARN (format: arn:aws:bedrock-agentcore:region:account:runtime/runtime-id)
runtime_id = agent_arn.split(":")[-1].split("/")[-1]
print(f"Runtime ID: {runtime_id}")

✅ MemoryManager initialized for region: us-east-1
🔎 Retrieving memory resource with ID: customer_support_agent_mem-k3d2NyGqvU...
  ✅ Found memory: customer_support_agent_mem-k3d2NyGqvU
Retrieved Bedrock AgentCore status for: customer_support_agent


Runtime ID: customer_support_agent-l62Ut2AD7M


In [11]:
import lab_helpers.utils as utils
import inspect

print("Available functions:")
for name, func in inspect.getmembers(utils, inspect.isfunction):
    if not name.startswith('_'):
        print(f"  - {name}")

Available functions:
  - agentcore_memory_cleanup
  - cleanup_cognito_resources
  - create_agentcore_runtime_execution_role
  - delete_agentcore_runtime_execution_role
  - delete_customer_support_secret
  - delete_observability_resources
  - delete_ssm_parameter
  - gateway_target_cleanup
  - get_aws_account_id
  - get_aws_region
  - get_cognito_client_secret
  - get_customer_support_secret
  - get_or_create_cognito_pool
  - get_ssm_parameter
  - load_api_spec
  - local_file_cleanup
  - put_ssm_parameter
  - read_config
  - reauthenticate_user
  - runtime_resource_cleanup
  - save_customer_support_secret


In [12]:
# Check if access_token exists
if 'access_token' in globals():
    print("✅ Token exists!")
    print(f"Keys: {access_token.keys()}")
else:
    print("❌ Token not found. You need to run the cell that creates it.")

✅ Token exists!
Keys: dict_keys(['pool_id', 'client_id', 'client_secret', 'secret_hash', 'bearer_token', 'discovery_url'])


In [13]:
import lab_helpers.utils as utils
import inspect

print("Available functions in lab_helpers.utils:")
print("="*70)

# Get all functions
members = inspect.getmembers(utils, inspect.isfunction)

for name, func in members:
    if not name.startswith('_'):
        # Get function signature
        try:
            sig = inspect.signature(func)
            print(f"\n{name}{sig}")
            # Get docstring if available
            if func.__doc__:
                doc = func.__doc__.strip().split('\n')[0]
                print(f"  Description: {doc}")
        except:
            print(f"\n{name}()")

print("\n" + "="*70)
print("\nSearching for token/auth related functions...")
print("="*70)

for name, func in members:
    if any(keyword in name.lower() for keyword in ['token', 'auth', 'cognito', 'access', 'bearer']):
        print(f"✅ Found: {name}")
        try:
            sig = inspect.signature(func)
            print(f"   Signature: {name}{sig}")
        except:
            print(f"   Signature: {name}()")

Available functions in lab_helpers.utils:

agentcore_memory_cleanup(memory_id: str = None)
  Description: List all memories and their associated strategies

cleanup_cognito_resources(pool_id)
  Description: Delete Cognito resources including users, app clients, and user pool

create_agentcore_runtime_execution_role()

delete_agentcore_runtime_execution_role()

delete_customer_support_secret()
  Description: Delete a secret from AWS Secrets Manager.

delete_observability_resources()

delete_ssm_parameter(name: str) -> None

gateway_target_cleanup(gateway_id: str = None)

get_aws_account_id() -> str

get_aws_region() -> str

get_cognito_client_secret() -> str

get_customer_support_secret()
  Description: Get a secret value from AWS Secrets Manager.

get_or_create_cognito_pool(refresh_token=False)

get_ssm_parameter(name: str, with_decryption: bool = True) -> str

load_api_spec(file_path: str) -> list

local_file_cleanup()

put_ssm_parameter(name: str, value: str, parameter_type: str = 'S

In [14]:
import boto3
from lab_helpers.utils import get_ssm_parameter

print("Fixing Cognito configuration...")

# Get configuration
client_id = get_ssm_parameter("/app/customersupport/agentcore/client_id")
cognito_discovery = get_ssm_parameter("/app/customersupport/agentcore/cognito_discovery_url")
pool_id = cognito_discovery.split('/')[3]

cognito_client = boto3.client('cognito-idp')

# Create resource server
try:
    cognito_client.create_resource_server(
        UserPoolId=pool_id,
        Identifier='bedrock-agentcore',
        Name='BedrockAgentCore',
        Scopes=[{'ScopeName': 'invoke', 'ScopeDescription': 'Invoke agent endpoints'}]
    )
    print("✅ Resource server created")
except Exception as e:
    if 'ResourceServerAlreadyExists' in str(e):
        print("✅ Resource server already exists")

# Update app client
response = cognito_client.describe_user_pool_client(
    UserPoolId=pool_id,
    ClientId=client_id
)

cognito_client.update_user_pool_client(
    UserPoolId=pool_id,
    ClientId=client_id,
    AllowedOAuthFlows=['client_credentials'],
    AllowedOAuthScopes=['bedrock-agentcore/invoke'],
    AllowedOAuthFlowsUserPoolClient=True,
    ExplicitAuthFlows=response['UserPoolClient'].get('ExplicitAuthFlows', []),
    SupportedIdentityProviders=['COGNITO']
)

print("✅ Configuration updated!")

Fixing Cognito configuration...
✅ Resource server created
✅ Configuration updated!


In [15]:
import boto3
import random
import string
from lab_helpers.utils import get_ssm_parameter

print("=" * 70)
print("FIXING ALL LAB 4 AUTHENTICATION ISSUES")
print("=" * 70)

# Get configuration
client_id = get_ssm_parameter("/app/customersupport/agentcore/client_id")
cognito_discovery = get_ssm_parameter("/app/customersupport/agentcore/cognito_discovery_url")
pool_id = cognito_discovery.split('/')[3]

cognito_client = boto3.client('cognito-idp')

# Fix 1: Create resource server
print("\n[1] Creating resource server...")
try:
    cognito_client.create_resource_server(
        UserPoolId=pool_id,
        Identifier='bedrock-agentcore',
        Name='BedrockAgentCore',
        Scopes=[{'ScopeName': 'invoke', 'ScopeDescription': 'Invoke agent endpoints'}]
    )
    print("✅ Resource server created")
except Exception as e:
    if 'ResourceServerAlreadyExists' in str(e):
        print("✅ Resource server already exists")

# Fix 2: Update app client
print("\n[2] Updating app client...")
response = cognito_client.describe_user_pool_client(UserPoolId=pool_id, ClientId=client_id)
cognito_client.update_user_pool_client(
    UserPoolId=pool_id,
    ClientId=client_id,
    AllowedOAuthFlows=['client_credentials'],
    AllowedOAuthScopes=['bedrock-agentcore/invoke'],
    AllowedOAuthFlowsUserPoolClient=True,
    ExplicitAuthFlows=response['UserPoolClient'].get('ExplicitAuthFlows', []),
    SupportedIdentityProviders=['COGNITO']
)
print("✅ App client updated")

# Fix 3: Create domain (CRITICAL!)
print("\n[3] Checking/creating domain...")
pool_response = cognito_client.describe_user_pool(UserPoolId=pool_id)
domain = pool_response['UserPool'].get('Domain')

if not domain:
    random_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=8))
    domain = f'customer-support-{random_suffix}'
    try:
        cognito_client.create_user_pool_domain(Domain=domain, UserPoolId=pool_id)
        print(f"✅ Domain created: {domain}")
    except:
        random_suffix = ''.join(random.choices(string.ascii_lowercase + string.digits, k=12))
        domain = f'cs-agent-{random_suffix}'
        cognito_client.create_user_pool_domain(Domain=domain, UserPoolId=pool_id)
        print(f"✅ Domain created: {domain}")
else:
    print(f"✅ Domain exists: {domain}")

print("\n✅ ALL FIXES APPLIED! Run next cell to get token.")

FIXING ALL LAB 4 AUTHENTICATION ISSUES

[1] Creating resource server...

[2] Updating app client...
✅ App client updated

[3] Checking/creating domain...
✅ Domain created: customer-support-pdh9bdyf

✅ ALL FIXES APPLIED! Run next cell to get token.


In [16]:
import boto3
import requests
import base64
from lab_helpers.utils import get_ssm_parameter, get_cognito_client_secret

print("Getting access token...")

# Get credentials
client_id = get_ssm_parameter("/app/customersupport/agentcore/client_id")
client_secret = get_cognito_client_secret()
cognito_discovery = get_ssm_parameter("/app/customersupport/agentcore/cognito_discovery_url")
pool_id = cognito_discovery.split('/')[3]
region = boto3.session.Session().region_name

# Get domain (works now after Fix 3!)
cognito_client = boto3.client('cognito-idp')
pool_response = cognito_client.describe_user_pool(UserPoolId=pool_id)
domain = pool_response['UserPool']['Domain']

# Build token URL
token_url = f"https://{domain}.auth.{region}.amazoncognito.com/oauth2/token"

# Request token
auth_string = f"{client_id}:{client_secret}"
auth_base64 = base64.b64encode(auth_string.encode('utf-8')).decode('utf-8')

response = requests.post(
    token_url,
    headers={
        'Content-Type': 'application/x-www-form-urlencoded',
        'Authorization': f'Basic {auth_base64}'
    },
    data={
        'grant_type': 'client_credentials',
        'scope': 'bedrock-agentcore/invoke'
    }
)

if response.status_code == 200:
    token_data = response.json()
    
    # CRITICAL: Create DICTIONARY structure (fixes ERROR 2!)
    access_token = {
        'bearer_token': token_data['access_token'],  # The actual token
        'client_id': client_id,
        'client_secret': client_secret,
        'pool_id': pool_id,
        'discovery_url': cognito_discovery
    }
    
    print("✅ Token obtained!")
    print(f"   Expires in: {token_data.get('expires_in')} seconds")
    print(f"   Token: {access_token['bearer_token'][:50]}...")
else:
    print(f"❌ Error: {response.json()}")
    raise Exception("Failed to get token")

Getting access token...
✅ Token obtained!
   Expires in: 3600 seconds
   Token: eyJraWQiOiJObHRmcU1telFNa1czM0x1dUZZbjhTRkV4R29xVm...


In [18]:
import uuid

# Create a session ID for demonstrating session continuity
session_id = uuid.uuid4()

# Test different customer support scenarios
user_query = "List all of your tools"

response = agentcore_runtime.invoke(
    {"prompt": user_query},
    bearer_token=access_token["bearer_token"],
    session_id=str(session_id),
)

print(response["response"])

"I'll be happy to help you with your electronics needs. Here are the tools I have available:\n\n1. `get_product_info` - Gets detailed technical specifications and information for electronics products\n2. `get_return_policy` - Gets return policy information for a specific product category\n3. `get_technical_support` - Provides technical support information for specific issues\n4. `LambdaUsingSDK___check_warranty_status` - Checks warranty status using a serial number\n5. `LambdaUsingSDK___web_search` - Searches the web for updated information\n\nBased on your preferences, it seems you're interested in gaming headphones with low latency for competitive FPS games, and you also have a MacBook Pro with overheating issues during video editing. How may I assist you today? Would you like information about gaming headphones, help with your MacBook Pro overheating issue, or something else?"


#### Invoking the agent with session continuity

Since we are using AgentCore Runtime, we can easily continue our conversation with the same session id.

In [19]:
user_query = "Tell me detailed information about the technical documentation on installing a new CPU"
response = agentcore_runtime.invoke(
    {"prompt": user_query},
    bearer_token=access_token["bearer_token"],
    session_id=str(session_id),
)
print(response["response"])

"Based on the search results, I can provide you with detailed information about installing a new CPU. Here's a comprehensive guide:\n\n## Installing a New CPU: Technical Documentation\n\n### Pre-Installation Preparation:\n\n1. **Compatibility Check**: \n   - Ensure the new CPU is compatible with your motherboard socket type\n   - Verify BIOS support for the new CPU model\n   - Check power supply capabilities for higher-end CPUs\n\n2. **Gather Required Tools**:\n   - Anti-static wrist strap\n   - Thermal paste\n   - Isopropyl alcohol and lint-free cloth (for cleaning old thermal paste)\n   - Phillips-head screwdriver\n   - CPU cooler (stock or aftermarket)\n\n### Installation Process:\n\n1. **Safety First**:\n   - Power down the system completely\n   - Unplug all power cables\n   - Press power button to discharge residual electricity\n   - Use anti-static measures (wrist strap, work on non-carpeted surface)\n\n2. **Accessing the CPU Socket**:\n   - Remove side panel of computer case\n  

#### Invoking the agent with a new user
In the example below we have not mentioned the Iphone device in the second query, but our agent still has the context of it. This is due to the AgentCore Runtime session continuity. The agent won't know the context for a new user.

In [20]:
# Creating a new session ID for demonstrating new customer
session_id2 = uuid.uuid4()

user_query = "I have a Gaming Console Pro device , I want to check my warranty status, warranty serial number is MNO33333333."
response = agentcore_runtime.invoke(
    {"prompt": user_query},
    bearer_token=access_token["bearer_token"],
    session_id=str(session_id2),
)
print(response["response"])

"Thank you for providing your serial number. Here's what I found about your Gaming Console Pro warranty:\n\n## Warranty Status Information\n- **Product**: Gaming Console Pro\n- **Serial Number**: MNO33333333\n- **Purchase Date**: November 25, 2023\n- **Warranty End Date**: November 25, 2024\n- **Warranty Type**: Gaming Warranty\n- **Status**: Expired (expired 436 days ago)\n\n### Coverage Details (when warranty was active):\n- Controller issues\n- Overheating protection\n- Hard drive replacement\n\nI'm sorry to inform you that your warranty has expired. However, you might be interested in extended warranty options that may still be available for your device.\n\nWould you like me to provide you with information about our extended warranty options for gaming consoles? Or perhaps you need technical support for an issue with your Gaming Console Pro? I also notice you're interested in gaming headphones with low latency for competitive FPS games - I'd be happy to help with that if you're loo

In this case our agent does not have the context anymore and needs more information. 

And it is all it takes to have a secure and scalable endpoint for our Agent with no need to manage all the underlying infrastructure!

### Step 5: AgentCore Observability

[AgentCore Observability](https://docs.aws.amazon.com/bedrock-agentcore/latest/devguide/observability.html) provides monitoring and tracing capabilities for AI agents using Amazon OpenTelemetry Python Instrumentation and Amazon CloudWatch GenAI Observability.

#### Agents

Default AgentCore Runtime configuration allows for logging our agent's traces on CloudWatch by means of **AgentCore Observability**. These traces can be seen on the AWS CloudWatch GenAI Observability dashboard. Navigate to Cloudwatch &rarr; GenAI Observability &rarr; Bedrock AgentCore.

![Agents Overview on CloudWatch](images/observability_agents.png)

#### Sessions

The Sessions view shows the list of all the sessions associated with all agents in your account.

![sessions](images/sessions_lab5_observability.png)

#### Traces

Trace view lists all traces from your agents in this account. To work with traces:

- Choose Filter traces to search for specific traces.
- Sort by column name to organize results.
- Under Actions, select Logs Insights to refine your search by querying across your log and span data or select Export selected traces to export.

![traces](images/traces_lab4_observability.png)


### Congratulations! 🎉

You have successfully completed **Lab 4: Deploy to Production - Use AgentCore Runtime with Observability!**

Here is what you accomplished:

##### Production-Ready Deployment:

- Prepared your agent for production with minimal code changes (only 4 lines added)
- Validated proper session isolation between different customers
- Confirmed session continuity + memory persistence and context awareness per session

##### Enterprise-Grade Security & Identity:

- Implemented secure authentication using Cognito integration with JWT tokens
- Configured proper IAM roles and execution permissions for production workloads
- Established identity-based access control for secure agent invocation

##### Comprehensive Observability:

- Enabled AgentCore Observability for full request tracing across all customer sessions
- Configured CloudWatch GenAI Observability dashboard monitoring

##### Current Limitations (We'll fix these next!):

- **Developer Focused Interaction** - Agent accessible via SDK/API calls but no user-friendly web interface
- **Manual Session Management** - Requires programmatic session creation rather than intuitive user experience

##### Next Up [Lab 5: Build User Interface →](lab-05-frontend.ipynb)
In Lab 5, you'll complete the customer experience by building a user-friendly interface !! Lets go !!
